In [ ]:
import savReaderWriter as spss
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import matplotlib.style as style
style.use('seaborn')

import ipywidgets as widgets
from ipywidgets import interact
from ipywidgets import interactive, Layout, HTML

from sklearn.cluster import KMeans

import itertools

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore")


#import plotly.plotly as py
#import chart_studio as py
#from chart_studio.plotly import plot, iplot
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
raw_data = spss.SavReader('p29_data.sav', returnHeader = True)
raw_data_list = list(raw_data)
df = pd.DataFrame(raw_data_list)
with spss.SavReader('p29_data.sav', ioUtf8 = True) as reader:
    df = pd.DataFrame(reader.all(), columns = [s.decode('CP1252') for s in reader.header])
    
prom= df[[ 'WEIGHT', 'GENDER', 'AGER','AGE','E1', 'E2', 'E3', 'E4', 'E5', 'E6']].copy()
eq5d= df[[ 'WEIGHT', 'GENDER', 'AGE','AGER','E1', 'E2', 'E3', 'E4', 'E5', 'E6']].copy()
eq5d.columns = ['WEIGHT', 'GENDER', 'AGE','AGE GROUP' ,'MOBILITY', 'SELF CARE', 'USUAL ACTIVITY', 'PAIN', 'ANXIETY', 'HEALTH SCALE']



eq= eq5d[['MOBILITY', 'SELF CARE', 'USUAL ACTIVITY', 'PAIN', 'ANXIETY']].copy()
new = eq5d.copy()
new['GENERAL HEALTH'] = eq.sum(axis=1)
#eq5d.head(3)    

In [ ]:
X = widgets.Dropdown(
    value='HEALTH SCALE', description='X',
    options=eq5d.columns)

Y = widgets.Dropdown(
    options=eq5d.columns.tolist(), description='Y',
    value='MOBILITY')

color= widgets.Dropdown(
    options=eq5d.columns.tolist(),description='COLOR BY',
    value='ANXIETY')


@interact(X=X,Y=Y,color = color)
def show(X,Y,color):
    fig = px.scatter(new, x='HEALTH SCALE', y=Y, color=color, marginal_y="violin",
               marginal_x="box", trendline="ols", template="simple_white")
    fig.show()

In [ ]:

X = widgets.Dropdown(
    value='HEALTH SCALE', description='X',
    options=eq5d.columns)

Y = widgets.Dropdown(
    options=eq5d.columns.tolist(), description='Y',
    value='USUAL ACTIVITY')

Z = widgets.Dropdown(
    options=eq5d.columns.tolist(), description='Z',
    value='MOBILITY')

color= widgets.Dropdown(
    options=eq5d.columns.tolist(),description='COLOR BY',
    value='ANXIETY')


@interact(X=X,Y=Y,Z=Z,color = color)
def show(X,Y,Z,color):
    fig = px.scatter_3d(new, x = X, y = Y, z = Z, color = color)
    
    fig.show()

In [ ]:
x = widgets.Dropdown(
    value='AGE GROUP', description='X',
    options=eq5d.columns)

y = widgets.Dropdown(
    options=eq5d.columns.tolist(), description='Y',
    value='MOBILITY')

z= widgets.Dropdown(
    options=eq5d.columns.tolist(),description='COLOR BY',
    value='ANXIETY')


def change(change):
    with fig.batch_update():
        f2.data[0].x = eq5d[x.value].values
        f2.data[0].y = eq5d[y.value].values
        f2.data[0].z = eq5d[z.value].values


t1 =  go.Contour(
            y=eq5d['MOBILITY'],
            z=eq5d['ANXIETY'], # horizontal axis
            x=eq5d['AGE GROUP'],
            connectgaps=True,
            contours=dict(
                coloring ='heatmap',
                showlabels = False, # show labels on contours
                labelfont = dict( # label font properties
                    size = 12,
                    color = 'white',
                )
        ))

data=[t1]
layout = go.Layout(title='Contor')

figure = go.Figure(data=data, layout=layout)

f2 = go.FigureWidget(figure)
##new
details = HTML()
scatter = f2.data[0]
def hover_fn(trace, points,state):
    ind = points.point_inds[0]
    details.value = eq5d.iloc[ind].to_frame().to_html()
    #m = eq5d.iloc[ind].MOBILITY
    #s = eq5d.iloc[ind]['SELF CARE']
    #u = eq5d.iloc[ind]['USUAL ACTIVITY']
    #p = eq5d.iloc[ind].PAIN
    #d = eq5d.iloc[ind].ANXIETY
    #c = Quest(m,s,u,p,d)
    
scatter.on_hover(hover_fn) 


#end
x.observe(change, names="value")
y.observe(change, names="value")
z.observe(change, names="value")



widgets.VBox([x,y,z,f2,details])


In [ ]:
#new or state one [mobility]
#yes > break
mobility = ['Do u have a problem walking about today?',
              'Is it a slight problem ? ',
            'Hmmm, Moderate problem ? ',
            'Are you still able to walk about? ']
mobility_sc = [2,3,4,5]

#new or state one [selfCare]
#yes > break
selfCare = ['Do u have any problems with self care (e.g washing or dressing yourself) ? ',
              'Is it a slight problems ?',
            'Moderate problems? ',
            'Okay, Are you still able to wash or dress yourself?']
selfCare_sc = [2,3,4,5]

#new or state one [activity]
#yes > break
activity = ['Do u have problems performing usual activities (e.g. work, study, housework, family or leisure activities)? ',
              'Is it a slight problem ?',
            'Moderate problems? ',
            'Are you still able to perform any of your usual activities?']
activity_sc = [2,3,4,5]

#new or state one [pain]
#yes > break
pain = ['Do u have any pain or discomfort ?',
          'Is it a slight pain or discomfort ?',
        'Okay, On a scale of 1-3 (1 being the lowest and 3 being the highest) how to rate your pain or discomfort level?']
pain_sc = [2,3,4,5]


#new or state one[depression]
#yes > break
anxiety = ['Are u feeling anxious or depressed ?',
             'On a scale of 1-4 (1 being the lowest and 4 being the highest) how to rate your anxiety or depression level?']
anxiety_sc = [2,3,4,5]

In [ ]:
# state 3 [mobility]
#yes > break
mobility_3 = ['Do u feel better or worse or the same at walking about today?',
              'Do u have any problem walking about?' ,
            'Are you still able to walk about? ']

#state 3 [selfCare]
#yes > break
selfCare_3 = ['Are you having less problems with self care (e.g washing or dressing yourself) than yesterday?',
              'Do u have slight problems regarding it  ?',
            'Okay, Are you still able to wash or dress yourself?',
            'Do u have moderate or severe problems regarding it  ?']

#state 3 [activity]
#yes > break
activity_3 = ['Are you feeling doing usual activities (e.g. work, study..) is easier or harder or the same as yesterday?',
              'Do u have slight problems doing usual activities ?',
            'Are you still able to perform any of your usual activities?',]


#state 3 [pain]
#yes > break
pain_3 = ['Regarding pain, Are you feeling better than yesterday? ',
          'Do u have any pain or discomfort ?',
        'Okay, On a scale of 1-3 (1 being the lowest and 3 being the highest) how to rate your pain or discomfort level?']

#state 3 [depression]
#yes > break
anxiety_3 = [ 'On a scale of 1-5 (1 being the lowest and 5 being the highest) how to rate your anxiety or depression level today?']

In [ ]:
# state 4 [mobility]
#yes > break
mobility_4 = ['Do u feel better or worse or the same in walking about today?',
              'Good, Do u still have problems in walking about? ',
              'Slight or moderate problems ?']

#state 4 [selfCare]
#yes > break
selfCare_4 = ['Do u find it easier or harder or the same to wash or dress yourself ?',
              'Good, Do u still have any problems  wash or dress yourself? ',
              'Slight or moderate problems?']


#state 4 [activity]
#yes > break
activity_4 = ['Do u find it easier or harder or the same to do your usual activities today (e.g. work, study..)?',
              'Great, Do u still have any problems doing your usual activities?',
              'Slight or moderate problems?']

#state 4 [pain]
#yes > break
pain_4 = ['Regarding pain, Are you feeling better than yesterday? ',
          'Great, you do still have some pain or discomfort, right?',
          'Slight or moderate pain?',
        'same pain or worse?' ]

#state 4 [depression]
#yes > break
anxiety_4 = [ 'On a scale of 1-5 (1 being the lowest and 5 being the highest) how to rate your anxiety or depression level today?']

In [ ]:
# state 5 [mobility]
#yes > break
mobility_5 = ['Are you able to walk about a little today?',
              'Good, Do u still have problems in walking about? ',
              'Okay, On a scale of 1-3 (1 being the lowest and 3 being the highest) how to rate your problems in walking about?']

#state 5 [selfCare]
#yes > break
selfCare_5 = [ 'Are you able to wash or dress yourself today? ',
              'Great, On a scale of 1-4 (1 being the lowest and 4 being the highest) how do u find it hard to wash or dress yourself?']


#state 5 [activity]
#yes > break
activity_5 = ['Have you done any usual activities today or able to (e.g. work, study..)?',
              'Great, On a scale of 1-4 (1 being the lowest and 4 being the highest) how do u find it hard to do your usual activities?']

#state 5 [pain]
#yes > break
pain_5 = ['Regarding pain, Are you feeling better than yesterday? ',
          'Great, you do still have some pain or discomfort, right?',
        'Okay, On a scale of 1-3 (1 being the lowest and 3 being the highest) how to rate your pain or discomfort level?']

#state 5 [depression]
#yes > break
anxiety_5 = [ 'On a scale of 1-5 (1 being the lowest and 5 being the highest) how to rate your anxiety or depression level today?']
anxiety_sc = [2,3,4,5]

In [ ]:

def Quest(Mobility,SelfCare,Activity,Pain,Anxiety):

    
    if Mobility == 0 or Mobility == 1 or Mobility == 2 :           
            mob_questions = mobility
    if Mobility == 3 :
            mob_questions = mobility_3    
    if Mobility == 4 :
            mob_questions = mobility_4           
    if Mobility == 5 :
            mob_questions = mobility_5
                     
            
    if SelfCare == 0 or SelfCare == 1 or SelfCare == 2 :           
            sf_questions = selfCare
    if SelfCare == 3 :
            sf_questions = selfCare_3 
    if SelfCare == 4 :
            sf_questions = selfCare_4          
    if SelfCare == 5 :
            sf_questions = selfCare_4     
            
    if Activity == 0 or Activity == 1 or Activity == 2 :           
            act_questions = activity
    if Activity == 3 :
            act_questions = activity_3
    if Activity == 4 :
            act_questions = activity_4          
    if Activity == 5 :
            act_questions = activity_5      
            
            
            
    if Pain == 0 or Pain == 1 or Pain == 2 :           
            pn_questions = pain
    if Pain == 3 :
            pn_questions = pain_3
    if Pain == 4 :
            pn_questions = pain_4        
    if Pain == 5 :
            pn_questions = pain_5  
            
    if Anxiety == 0 or Anxiety == 1 or Anxiety == 2 :           
            ad_questions = anxiety
    if Anxiety== 3 :
            ad_questions = anxiety_3
    if Anxiety == 4 :
            ad_questions = anxiety_4          
    if Anxiety == 5 :
            ad_questions = anxiety_5    
            


    return mob_questions, sf_questions, act_questions, pn_questions, ad_questions
    


In [ ]:
from sklearn.decomposition import PCA

variance = 3
pca = PCA(variance)
pca.fit(new)
x = pca.transform(new)
WCSS = []
for i in range(1,11):
    model = KMeans(n_clusters = i,init = 'k-means++')
    model.fit(x)
    WCSS.append(model.inertia_)
#fig = plt.figure(figsize = (7,7))
#plt.plot(range(1,11),WCSS, linewidth=4, markersize=12,marker='o',color = 'red')
#plt.xticks(np.arange(11))
#plt.xlabel("Number of clusters")
#plt.ylabel("WCSS")
#plt.show()

In [ ]:

model = KMeans(n_clusters = 5, init = "k-means++", max_iter = 300, n_init = 10, random_state = 0)
y_clusters = model.fit_predict(x)

# 3d scatterplot using plotly
Scene = dict(xaxis = dict(title  = 'PCA 1'),yaxis = dict(title  = 'PCA 2'),zaxis = dict(title  = 'PCA 3 '))

# model.labels_ is nothing but the predicted clusters i.e y_clusters
labels = model.labels_
trace = go.Scatter3d(x=x[:, 0], y=x[:, 1], z=x[:, 2], mode='markers',marker=dict(color = labels, size= 10, line=dict(color= 'black',width = 10)))
layout = go.Layout(margin=dict(l=0,r=0),scene = Scene,height = 800,width = 800)
data = [trace]
clust = go.Figure(data = data, layout = layout)


clus = go.FigureWidget(clust)

details3 = HTML()
details2 = HTML()

scatt = clus.data[0]

def hover_fn(trace, points,state):
    ind = points.point_inds[0]
    details3.value = new.iloc[ind].to_frame().to_html()
    m = new.iloc[ind].MOBILITY
    s = new.iloc[ind]['SELF CARE']
    u = new.iloc[ind]['USUAL ACTIVITY']
    p = new.iloc[ind].PAIN
    d = new.iloc[ind].ANXIETY
    c = Quest(m,s,u,p,d)
    details2.value =pd.DataFrame(c).to_html()
    
scatt.on_hover(hover_fn) 

#clus.show()
wid = widgets.HBox([details3,details2])
widgets.VBox([clus,wid])